In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

In [2]:
from itertools import product, chain
from random import choices, shuffle

In [3]:
ALL_SUITS = list(range(5))
ALL_CARDS = [(x,y) for (x,y) in product(ALL_SUITS, range(9)) if x>0 or y<4]
ALL_TASKS = list(ALL_CARDS[4:])

In [4]:
CARD_INDICER = {x:i for i, x in enumerate(ALL_CARDS)}
TASK_INDICER = {x:i for i, x in enumerate(ALL_TASKS)}

In [5]:
N = 4
M = 4

In [11]:
DEBUG = None

In [105]:
class AssignerNet(nn.Module):
    def __init__(self, n):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Linear(len(TASK_INDICER)*(n+1)+n+len(CARD_INDICER), 64),
            nn.ReLU(),
            nn.Linear(64, 64),
            nn.ReLU(),
            nn.Linear(64, len(TASK_INDICER)),
            nn.Softmax(dim=1)
        )
        self.float()

    def forward(self, x):
        logits = self.layers(x)
        return logits

class Assigner:
    
    def __init__(self, n, m):
        self.n = n
        self.m = m
        self.nn = AssignerNet(n)
        self.loss_fn = nn.MSELoss().float()
    
    def assign(self, hands, tasks):
        lead = next(i for i in range(self.n) if (0,3) in hands[i])
        chooser = lead
        task_mask = np.zeros(len(TASK_INDICER), dtype=bool)
        task_assignment_masks = np.zeros((self.n, len(TASK_INDICER)), dtype=bool)
        lead_mask = np.zeros(self.n, dtype=bool)
        hand_masks = np.zeros((self.n, len(CARD_INDICER)), dtype=bool)
        
        self.nn.eval()
        
        for t in tasks:
            task_mask[TASK_INDICER[t]] = 1
        
        for i in range(self.n):
            for card in hands[i]:
                hand_masks[i][CARD_INDICER[card]] = 1

        for _ in range(self.m):
            lead_mask[(lead-chooser+self.n)%self.n] = 1
            inp = np.concatenate((
                task_mask,
                task_assignment_masks[chooser:].flatten(),
                task_assignment_masks[:chooser].flatten(),
                lead_mask,
                hand_masks[chooser]
            )).reshape((1, -1))
            with torch.no_grad():
                out = self.nn(torch.tensor(inp, dtype=torch.float)).detach().numpy().flatten()
            out[~task_mask] = 0.
            out[task_assignment_masks.sum(axis=0).astype(bool)] = 0.
            # print(out)
            # print(task_mask.astype(int))
            # print(task_assignment_masks.astype(int))
            # print(task_assignment_masks.sum(axis=0).astype(int))
            # print()
            k = np.argmax(out)
            task_assignment_masks[chooser][k] = 1
            lead_mask[(lead-chooser+self.n)%self.n] = 0
            chooser = (chooser+1)%self.n
        
        return task_assignment_masks

    def learn_from(self, hands, tasks, task_assignment_masks, won_cards_mask):
        lead = next(i for i in range(self.n) if (0,3) in hands[i])
        chooser = lead
        task_mask = np.zeros(len(TASK_INDICER), dtype=bool)
        task_assignment_masks_partial = np.zeros((self.n, len(TASK_INDICER)), dtype=bool)
        lead_mask = np.zeros(self.n, dtype=bool)
        hand_masks = np.zeros((self.n, len(CARD_INDICER)), dtype=bool)
        
        for t in tasks:
            task_mask[TASK_INDICER[t]] = 1
        
        for i in range(self.n):
            for card in hands[i]:
                hand_masks[i][CARD_INDICER[card]] = 1

        for _ in range(self.m):
            lead_mask[(lead-chooser+self.n)%self.n] = 1
            inp = np.concatenate((
                task_mask,
                task_assignment_masks_partial[chooser:].flatten(),
                task_assignment_masks_partial[:chooser].flatten(),
                lead_mask,
                hand_masks[chooser]
            )).reshape((1, -1))
            out = self.nn(torch.tensor(inp, dtype=torch.float)).flatten()
            target = (won_cards_mask[chooser][4:] & task_mask)
            target = torch.tensor(target/(1e-6 + target.sum()), dtype=torch.float)
            loss = self.loss_fn(out, target)
            loss.backward()
            # print(out)
            # print(task_mask.astype(int))
            # print(task_assignment_masks.astype(int))
            # print(task_assignment_masks.sum(axis=0).astype(int))
            # print()

            task_assignment_masks_partial[chooser] = task_assignment_masks[chooser]
            lead_mask[(lead-chooser+self.n)%self.n] = 0
            chooser = (chooser+1)%self.n
        
        return task_assignment_masks_partial

In [106]:
# class PlayerNet(nn.Module):
#     def __init__(self, n):
#         super().__init__()
#         self.layers = nn.Sequential(
#             nn.Linear(len(TASK_INDICER)*(n+1)+n+len(CARD_INDICER), 512),
#             nn.ReLU(),
#             nn.Linear(512, 512),
#             nn.ReLU(),
#             nn.Linear(512, len(CARD_INDICER)),
#             nn.Sigmoid(),
#         )

#     def forward(self, x):
#         raise NotImplementedError
#         return self.layers(x)

# class Player:
    
#     def __init__(self, n, m):
#         self.n = n
#         self.m = m
#         self.nn = PlayerNet(n)
    
#     def assign(self, hands, tasks):
#         raise NotImplementedError
#         lead = next(i for i in range(self.n) if (0,3) in hands[i])
#         chooser = lead
#         task_mask = np.zeros(len(TASK_INDICER), dtype=bool)
#         task_assignment_masks = np.zeros((self.n, len(TASK_INDICER)), dtype=bool)
#         lead_mask = np.zeros(self.n, dtype=bool)
#         hand_masks = np.zeros((self.n, len(CARD_INDICER)), dtype=bool)

#         for t in tasks:
#             task_mask[TASK_INDICER[t]] = 1

#         for i in range(self.n):
#             for card in hands[i]:
#                 hand_masks[i][CARD_INDICER[card]] = 1

#         for _ in range(self.m):
#             lead_mask[(lead-chooser+self.n)%self.n] = 1
#             inp = np.concatenate((
#                 task_mask,
#                 task_assignment_masks[chooser:].flatten(),
#                 task_assignment_masks[:chooser].flatten(),
#                 lead_mask,
#                 hand_masks[chooser]
#             ))
#             out = self.nn(torch.tensor(inp, dtype=torch.float)).detach().numpy().flatten()
#             out[~task_mask] = 0.
#             out[task_assignment_masks.sum(axis=0).astype(bool)] = 0.
#             # print(out)
#             # print(task_mask.astype(int))
#             # print(task_assignment_masks.astype(int))
#             # print(task_assignment_masks.sum(axis=0).astype(int))
#             # print()
#             k = np.argmax(out)
#             task_assignment_masks[chooser][k] = 1
#             lead_mask[(lead-chooser+self.n)%self.n] = 0
#             chooser = (chooser+1)%self.n

#         return task_assignment_masks

In [107]:
class Game:
    def __init__(self, n, m, assigner=None):
        self.n = n
        self.m = m
        self.tasks = choices(ALL_TASKS, k=m)
        self.initialize_hands()
        self.lead = next(i for i in range(self.n) if (0,3) in self.hands[i])
        self.assigner = assigner if assigner else Assigner(self.n, self.m)
    
    def assign_tasks(self):
        self.task_assignment_masks = self.assigner.assign(self.hands, self.tasks)
        # print(self.task_assignment_masks.astype(int))
        # print()
    
    def initialize_hands(self):
        deck = list(ALL_CARDS)
        shuffle(deck)
        self.hands = []
        self.initial_hands = []
        for i in range(self.n):
            self.hands.append(deck[i*(40//self.n):(i+1)*(40//self.n)])
            self.initial_hands.append(deck[i*(40//self.n):(i+1)*(40//self.n)])
    
    def play_randomly(self):
        self.won_cards = [[] for _ in range(self.n)]
        self.task_completion_masks = np.zeros((self.n, len(TASK_INDICER)), dtype=bool)
        lead = self.lead
        for _ in range(40//self.n):
            trick = choices(self.hands[lead])
            # print(trick, self.hands[lead])
            self.hands[lead].remove(trick[0])
            w = 0
            for j in range(1, self.n):
                options = [c for c in self.hands[(lead+j)%self.n] if c[0]==trick[0][0]]
                if not options:
                    options = self.hands[(lead+j)%self.n]
                trick.extend(choices(options))
                self.hands[(lead+j)%self.n].remove(trick[-1])
                if (trick[j][0] == trick[w][0] and trick[j][1] > trick[w][1]) or (trick[j][0] == 0 < trick[w][0]):
                    w = j
            w = (w + lead) % self.n
            self.won_cards[w].extend(trick)
            for c in trick:
                if c[0] > 0:
                    t = TASK_INDICER[c]
                    self.task_completion_masks[w][t] = self.task_assignment_masks[:, t].sum()
            lead = w
        
        self.score = (self.task_assignment_masks & self.task_completion_masks).sum() / self.m
        # print(self.task_assignment_masks.astype(int))
        # print(self.task_completion_masks.astype(int))
        # print()

    def teach_assigner(self):
        won_cards_mask = np.zeros((self.n, len(CARD_INDICER)), dtype=bool)
        for i in range(self.n):
            for card in self.won_cards[i]:
                won_cards_mask[i, CARD_INDICER[card]] = 1
        
        self.assigner.learn_from(self.initial_hands, self.tasks, self.task_assignment_masks, won_cards_mask)

#         assert sorted(list(chain(*self.hands)) + list(chain(*self.won_cards))) == ALL_CARDS, (sorted(list(chain(*self.hands))), ALL_CARDS, self.hands)


In [108]:
assigner = None

In [115]:
games = []
for _ in range(1000):
    game = Game(N, M, assigner=assigner)
    assigner = game.assigner
    game.assign_tasks()
    game.play_randomly()
    game.teach_assigner()
    games.append(game)

scores = [game.score for game in games]

In [116]:
print(np.mean(scores))

0.23675
